In [1]:
import sys
sys.path.insert(1, './scripts')

from libraries import *
from functions import *
from fitting import *
from util import *

%matplotlib qt
data[0]

/home/alanyu/anaconda3/lib/python3.8/site-packages/pysis/env.py:33: RuntimeWarning: Could not find isis. Is `ISISROOT` set?
  warnings.warn('Could not find isis. Is `ISISROOT` set?', RuntimeWarning)


Current progress: 50000
Current progress: 100000
Current progress: 150000
Current progress: 200000
Current progress: 250000
DONE


{'name': 'V1649180633_1.ir_rc20final.cub',
 'sequenceid': 'S59',
 'observationid': 'VIMS_129TI_REGMAP001_ISS',
 'latitude': 6.527894053385,
 'longitude': 117.23227585525,
 'phase': 4.35186034428,
 'incidence': 71.368914431631,
 'emission': 72.074559422824,
 'distance': 25208.69808185,
 'lineres': 38679.377390385,
 'sampres': 12885.433750399,
 'scposx': 15904.022205055,
 'scposy': 19558.581460496,
 'scposz': 49.274092100503,
 'sunposx': 993649050.83181,
 'sunposy': 1013825175.7289,
 'sunposz': 93586120.333315,
 'ettime': 323758866.57842,
 'starttime': '2010-04-05 16:59:19.48',
 'exposureduration': 220.0,
 'spectrum': array([ 0.0763505 ,  0.113136  ,  0.138922  ,  0.146524  ,  0.126     ,
         0.0936954 ,  0.0795553 ,  0.0763359 ,  0.0885651 ,  0.111927  ,
         0.131274  ,  0.140332  ,  0.13276   ,  0.100027  ,  0.0593369 ,
         0.0431969 ,  0.0295527 ,  0.0313845 ,  0.042422  ,  0.0622391 ,
         0.0827431 ,  0.0844537 ,  0.0981416 ,  0.109633  ,  0.124524  ,
         0.0

### Why are some pixels unreasonable?


There a few exploding spectra but a few that dip under 0. Verified that it is consistent with CSV output from SQL query. Work around this by filtering pixels out according to fit parameters and raw spectra values.

In [8]:
bad_pixels = []
filtered_data = filter_data(data)


for i,entry in enumerate(filtered_data):
    if np.max(entry['spectrum']) > 1.0 or np.min(entry['spectrum']) < -.1:
        bad_pixels.append((entry,i))

len(bad_pixels)

{'name': 'V1753535313_1.ir_rc20final.cub', 'sequenceid': 'S79', 'observationid': 'VIMS_195TI_HIRES003_PRIME', 'latitude': -14.192410526012, 'longitude': 113.08447714133, 'phase': 79.247331641247, 'incidence': 68.628679969023, 'emission': 26.016340587574, 'distance': 7368.7025357152, 'lineres': 1965.8545564098, 'sampres': 2466.4575678517, 'scposx': -2735.0425878636, 'scposy': 5677.0692141084, 'scposz': -3819.4507773567, 'sunposx': -1380094499.4225, 'sunposy': 147413363.40772, 'sunposz': 488789535.78307, 'ettime': 428112953.54426, 'starttime': '2013-07-26 12:12:41.994', 'exposureduration': 160.0, 'spectrum': array([ 0.0404514 ,  0.0663658 ,  0.086725  ,  0.0854239 ,  0.0644469 ,
        0.0462576 ,  0.0390803 ,  0.0374057 ,  0.048135  ,  0.0629884 ,
        0.0766593 ,  0.0830114 ,  0.0691124 ,  0.0468681 ,  0.0240458 ,
        0.0202946 ,  0.0128574 ,  0.016177  ,  0.021298  ,  0.032576  ,
        0.0397113 ,  0.0422687 ,  0.0508583 ,  0.059501  ,  0.0663442 ,
        0.042854  ,  0.026

Total 94816 entries removed


0

In [4]:
def plot_spectra(entry):
    plt.plot(vims_wave, entry)

In [5]:
plot_spectra(filtered_data[32131]['spectrum'])

### Plot bump parameters against viewing geometry

Looking for empirical corrections that can be applied.

In [31]:
def plot_bump_parameters(data, sample_every = 1):
    
    from mpl_toolkits.mplot3d import Axes3D

    # should also consider filtering out entries where bump isn't present? 
    
    fig, axs = plt.subplots(3,3)
    data_as_list = {'incidence': [], 
                    'emission': [], 
                    'phase': [], 
                    'bump_max': [],
                    'bump_max_channel': [],
                    'slope': [],
                    'constant': [],
                    'ettime': []}
    
    for i, entry in enumerate(data):
        if i % sample_every == 0 and entry['bump_info'] != {}:
            
            data_as_list['incidence'].append(entry['incidence'])
            data_as_list['emission'].append(entry['emission'])
            data_as_list['phase'].append(entry['phase'])
            data_as_list['bump_max'].append(entry['bump_info']['bump_max'])
            data_as_list['slope'].append(entry['bump_info']['polyfit'][0])
            data_as_list['constant'].append(entry['bump_info']['polyfit'][1])
            data_as_list['bump_max_channel'].append(entry['bump_info']['bump_max_channel'])
            data_as_list['ettime'].append(entry['ettime'])
            
    axs[0, 0].scatter(data_as_list['incidence'], data_as_list['bump_max'])
    axs[0, 0].set_title('Bump max vs Incidence')
    axs[0, 0].set_ylim([0,.02])
    
    axs[0, 1].scatter(data_as_list['emission'], data_as_list['bump_max'])
    axs[0, 1].set_title('Bump max vs Emission')
    axs[0, 1].set_ylim([0,.02])

    axs[0, 2].scatter(data_as_list['phase'], data_as_list['bump_max'])
    axs[0, 2].set_title('Bump max vs Phase')
    axs[0, 2].set_ylim([0,.02])

#     axs[1, 0].scatter(data_as_list['incidence'], data_as_list['bump_max_channel'])
#     axs[1, 0].set_title('Bump channel vs Incidence')

#     axs[1, 1].scatter(data_as_list['emission'], data_as_list['bump_max_channel'])
#     axs[1, 1].set_title('Bump channel vs Emission')

#     axs[1, 2].scatter(data_as_list['phase'], data_as_list['bump_max_channel'])
#     axs[1, 2].set_title('Bump channel vs Phase')

    axs[1, 0].scatter(data_as_list['incidence'], data_as_list['slope'])
    axs[1, 0].set_title('Bump slope vs Incidence')
    
    axs[1, 1].scatter(data_as_list['emission'], data_as_list['slope'])
    axs[1, 1].set_title('Bump slope vs Emission')
    
    axs[1, 2].scatter(data_as_list['phase'], data_as_list['slope'])
    axs[1, 2].set_title('Bump slope vs Phase')
    
    axs[2, 0].scatter(data_as_list['incidence'], data_as_list['constant'])
    axs[2, 0].set_title('Bump constant vs Incidence')
    
    axs[2, 1].scatter(data_as_list['emission'], data_as_list['constant'])
    axs[2, 1].set_title('Bump constant vs Emission')
    
    axs[2, 2].scatter(data_as_list['phase'], data_as_list['constant'])
    axs[2, 2].set_title('Bump Constant vs Phase')
    
    plt.tight_layout(pad=0.00001)

    fig2 = plt.figure()
    ax = fig2.add_subplot(111, projection = '3d')
    
    ax.scatter(data_as_list['bump_max'], data_as_list['incidence'], data_as_list['emission'])
    ax.set_xlabel('bump max')
    ax.set_ylabel('incidence')
    ax.set_zlabel('emission')
    
    fig3 = plt.figure()
    ax3 = fig3.add_subplot(111)
    ax3.set_title('Bump max vs Time')
    ax3.scatter(data_as_list['bump_max'], data_as_list['ettime'])

In [7]:
filtered_data = list(filter(lambda x: 20 > x['emission'] > 10 and 20 > x['incidence'] > 10, filtered_data))

In [32]:
plot_bump_parameters(filtered_data, 10)

In [33]:
high_bumps = []

for entry in filtered_data:
    if entry['bump_info']['bump_max'] > .04:
        high_bumps.append(entry)
        


In [39]:
high_bumps

[{'name': 'V1582414888_1.ir_rc20final.cub',
  'sequenceid': 'S38',
  'observationid': 'VIMS_059TI_GLOBMAP001',
  'latitude': 3.1067470947883,
  'longitude': 128.86660634075,
  'phase': 65.714350518166,
  'incidence': 60.012796056184,
  'emission': 23.007649005019,
  'distance': 119171.06942272,
  'lineres': 62296.965776014,
  'sampres': 58427.474271559,
  'scposx': -69223.671482183,
  'scposy': 82210.178826049,
  'scposz': 51490.907850521,
  'sunposx': -1360015551.9193,
  'sunposy': -190753014.95976,
  'sunposz': -195089876.50793,
  'ettime': 256993581.50891,
  'starttime': '2008-02-22 23:04:48.661',
  'exposureduration': 80.0,
  'spectrum': array([ 0.0453567 ,  0.068492  ,  0.0948523 ,  0.11213   ,  0.0866385 ,
          0.0578778 ,  0.0457019 ,  0.0432139 ,  0.049996  ,  0.0659662 ,
          0.085769  ,  0.0991651 ,  0.0953957 ,  0.0608464 ,  0.0359847 ,
          0.0235448 ,  0.0179257 ,  0.016657  ,  0.0226608 ,  0.0321711 ,
          0.0439293 ,  0.0444438 ,  0.0645188 ,  0.06454

In [43]:
plt.plot(vims_wave, high_bumps[8]['spectrum'])

In [40]:
len(high_bumps)

27

In [37]:
print([(x['name'], x['latitude'], x['longitude']) for x in high_bumps])

[('V1552198125_1.ir_rc20final.cub', 5.1574220326716, 112.27068571033), ('V1559077376_1.ir_rc20final.cub', 7.3891657499115, 118.51583656536), ('V1567248530_1.ir_rc20final.cub', -4.5168224370617, 114.30216012076), ('V1664052654_1.ir_rc20final.cub', -8.217874483662, 101.8685658377), ('V1664051591_1.ir_rc20final.cub', -2.0888365030181, 109.16083003572), ('V1570025581_1.ir_rc20final.cub', 7.3081445724759, 119.32796898156), ('V1550837453_1.ir_rc20final.cub', -2.2921475060595, 112.0543135225), ('V1702525292_1.ir_rc20final.cub', -5.8289254301066, 177.87817011655), ('V1567243182_1.ir_rc20final.cub', -2.8997700724539, 170.49537813414)]


In [52]:
def examine_bump(i, start, end):
    fig, axs = plt.subplots(2,2)
    
    spectrum = filtered_data[i]['spectrum']
    p = np.polyfit(np.append(vims_wave[start[0]:start[1]],vims_wave[end[0]:end[1]]), np.append(spectrum[start[0]:start[1]],spectrum[end[0]:end[1]]), 1)
    fit = np.polyval(p, np.append(vims_wave[start[0]:start[1]], vims_wave[end[0]:end[1]]))
    
    axs[0,0].plot(np.append(vims_wave[start[0]:start[1]],vims_wave[end[0]:end[1]]), fit, label = 'Linear fit')
    axs[0,0].plot(vims_wave[start[0]:end[1]], filtered_data[i]['spectrum'][start[0]:end[1]], label = 'Spectra')
    axs[0,0].plot(vims_wave[60:85], spectrum[60:85])
    axs[0,0].plot(vims_wave[start[0]:start[1]], spectrum[start[0]:start[1]], color = 'r')
    axs[0,0].plot(vims_wave[end[0]:end[1]], spectrum[end[0]:end[1]], color = 'r')
    
    axs[0,1].plot(vims_wave[start[0]:end[1]], np.append(np.append(spectrum[start[0]:start[1]]-fit[:start[1]-start[0]],
                                                                  spectrum[start[1]:end[0]]-np.polyval(p, vims_wave[start[1]:end[0]])), 
                                                                  spectrum[end[0]:end[1]]-fit[start[1]-start[0]:]))
    axs[1,0].plot(vims_wave[60:85], filtered_data[i]['spectrum'][60:85])
    axs[1,0].plot(vims_wave[start[1]:end[0]], spectrum[start[1]:end[0]])
    
    axs[1,1].plot(vims_wave,spectrum)
    plt.legend()
    print(np.max(filtered_data[i]['spectrum']))
    
examine_bump(412, (70,72), (77,79))

No handles with labels found to put in legend.


0.143599


In [83]:
time_pixels = list(filter(lambda x: 2.425e8 > x['ettime'] > 2.4e8 and 0 < x['bump_info']['bump_max'] < .003, filtered_data))

In [80]:
len(time_pixels)

3030

In [81]:
print(len(np.unique([x['name'] for x in time_pixels])))

48


In [85]:
plt.scatter([x['longitude'] for x in time_pixels], [x['latitude'] for x in time_pixels], c = [x['bump_info']['bump_max'] for x in time_pixels])

In [89]:
len(filtered_data)

184976

In [14]:
def bin_data(data, remove = 0.05):
    new_data = sorted(data, key = lambda e: e['bump_info']['bump_max'], reverse = True)
    num_to_delete = int(len(data) * remove)
    return new_data[num_to_delete:]

In [15]:
new_data = bin_data(filtered_data)

In [23]:
def airmass(incidence, emission):
    return 1/(np.cos(np.radians(incidence))) + 1/(np.cos(np.radians(emission)))

In [87]:
def get_list(new_data, sample = 1):
    airmasses = []
    slopes = []
    constants = []
    
    for i, x in enumerate(new_data):
        if i%sample == 0:
            if airmass(x['incidence'], x['emission']) > 50 or abs(airmass(x['incidence'], x['emission']) - 2) < .5:
                continue
            else:
                airmasses.append(airmass(x['incidence'], x['emission']))
                slopes.append(x['bump_info']['polyfit'][0])
                constants.append(x['bump_info']['polyfit'][1])
        
    return airmasses, slopes, constants

airmasses, slopes, constants = get_list(new_data)
print(len(airmasses), len(new_data))

106517 175728


In [107]:
def plot_airmass_functions(airmasses, slopes, constants):
    fig, ax = plt.subplots(2, 2)
    ax[0,0].scatter(airmasses, constants, alpha = 0.3)
    ax[0, 0].set_ylabel('Constants')
    ax[0, 0].set_xlabel('Airmasses')
    constant_fit, cov = sco.curve_fit(powerlaw_no_constant, airmasses, constants, p0 = (1, -.5))
    print(constant_fit)
    fit = powerlaw_no_constant(np.linspace(min(airmasses), max(airmasses), 1000), *constant_fit)
    ax[0, 0].plot(np.linspace(min(airmasses), max(airmasses), 1000), fit, c = 'r')
    
    ax[0, 1].scatter(airmasses, slopes, alpha = 0.3)
    ax[0, 1].set_ylabel('Slopes')
    ax[0, 1].set_xlabel('Airmasses')
    
    plt.show()

plot_airmass_functions(airmasses, slopes, constants)

[ 0.83824522 -0.26354295]


<function functions.powerlaw(x, a, k, c)>

In [65]:
from scipy import stats
stats.mode(airmasses)[0]

array([2.0090867])

In [78]:
len([x for x in airmasses if  abs(x-2.2) < .01])

3775

In [67]:
airmasses

[2.4858538858508026,
 4.950740421157089,
 5.073874757889403,
 4.968519975363085,
 6.231657123388729,
 4.792064088472401,
 2.859615578317141,
 6.80744096149518,
 4.7400549619583,
 4.838014093851212,
 7.465406222628866,
 5.9669357360821325,
 4.852783728321349,
 2.8196570344412026,
 5.7108133985532215,
 4.023466013909321,
 4.505321269322565,
 5.87955961844653,
 4.654451614211868,
 5.621618469749032,
 2.1294717118584408,
 5.185498310720947,
 5.700928003857868,
 4.88540748150868,
 5.066403711486862,
 5.045044731581693,
 2.8305958576765295,
 5.143785197119021,
 7.679233376460478,
 5.001180968040519,
 2.3779651649582068,
 5.044664565590525,
 5.342935924349026,
 4.5081712367451345,
 4.984252713461215,
 7.640393410884515,
 5.876787119897072,
 6.1065306625128155,
 4.693144941046549,
 5.065839843075487,
 6.756114648546,
 6.306305522616558,
 5.623211291239114,
 7.499385027514537,
 4.818985690074122,
 5.656047497611144,
 5.394546737511793,
 5.170648891237962,
 7.419975789408184,
 4.968092947957621,